## 1.3 Retrieval (minsearch) 

- minsearch is a simple vector search library that allows you to index and search text documents. (https://github.com/alexeygrigorev/minsearch)
- extracting information from Google Docs and parse into a json file: https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/parse-faq.ipynb



In [1]:
#!pip install minsearch

In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'r') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields = ['question', 'text', 'section'],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp'; 

In [8]:
q = 'The course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
# when we think one of the text fields is more important then others, we can boost it 
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    # filter by course
    filter_dict = {'course': 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results=5
)

In [11]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## 1.4 Generation with OpenAI


In [12]:
from openai import OpenAI

In [13]:
from dotenv import load_dotenv

In [14]:
load_dotenv()

True

In [15]:
client = OpenAI()

In [16]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}],
)

In [17]:
response.choices[0].message.content

"Whether you can still enroll in a course after it has started depends on the specific policies of the institution or platform offering the course. Here are a few steps you can take to find out:\n\n1. **Check the Course Syllabus or Description:** Often, the enrollment deadlines are mentioned in these documents.\n\n2. **Contact the Instructor or Course Administrator:** Reach out directly to the person teaching the course or the administrative office. They may allow late enrollment, especially if the course is self-paced or has flexible deadlines.\n\n3. **Visit the Institution's Website:** Many institutions have detailed information about enrollment policies on their websites, including whether late enrollments are accepted.\n\n4. **Consider the Course Format:** Some courses, especially online or self-paced ones, have more flexible enrollment options, allowing students to join even after the official start date.\n\nIf late enrollment is possible, be prepared to catch up on any missed wor

In [18]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION. If the CONTEXT does not contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [19]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n" 

In [20]:
prompt = prompt_template.format(question=q, context=context).strip()

In [21]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION. If the CONTEXT does not contain the answer, output NONE.

QUESTION: The course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your 

In [22]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}],
)

response.choices[0].message.content

'Yes, you can still enroll in the course after it has started. You are eligible to submit the homework assignments even if you register late. However, be mindful of the deadlines for turning in the final projects and try not to leave everything for the last minute.'

## 1.5 Cleaned RAG flow and Modularizing Code



In [23]:
def search(query):
    # when we think one of the text fields is more important then others, we can boost it 
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        # filter by course
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results=5
    )

    return results

In [24]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION. If the CONTEXT does not contain the answer, output NONE.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()
    
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt


In [25]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

In [26]:
def rag(query):
    results = search(query)
    prompt = build_prompt(query, results)
    answer = llm(prompt)
    return answer

In [27]:
# not sure why this is giving NONE when it should work
query = 'how do i run kafka?'
rag(query)

'NONE'

In [28]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course even after it has started. You are eligible to submit the homework, but be mindful of the deadlines for turning in final projects.'

## 1.6 Search with Elasticsearch

- Run ElasticSearch with Docker
- Index the documents
- Replace MinSearch with ElasticSearch


- Elasticsearch is a search engine often used to index and search large amounts of text data quickly.


Run this in terminal:

```
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [29]:
from elasticsearch import Elasticsearch

Had to run this `pip install "elasticsearch<9.0.0"` to avoid error.

In [30]:
es_client = Elasticsearch('http://localhost:9200')

In [32]:
es_client.info()

ObjectApiResponse({'name': '797ebad56cbf', 'cluster_name': 'docker-cluster', 'cluster_uuid': '1-Bqbhg4SxSBxnZwVfMQcQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [51]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [36]:
from tqdm.auto import tqdm

In [37]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

- `"fields": ["question^3", "text", "section"],`: question is given a higher weight in the search results.

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [49]:
query = 'I just disovered the course. Can I still join it?'
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."